Calculate drive time/ OSRM server
Author: Jonas Krueger
last Edit: 01/26/18



In [55]:
import pyodbc
import pandas as pd
import os
import sys
import json
import time
import csv
import requests 
import asyncio
from aiohttp import ClientSession
import aiohttp
import async_timeout

The setup is based on an OSRM backend running on a (cloud) server. I use google cloud, but any setup will work.

1. Open connection to SQL server to load base table
    + Base table has all lang/lat pairs, for which we want to calculate drive time and distance
2. Since we don’t want to read in the complete table we looping through chunks of it
    + With `mod(VALUE,100)` we can split the base table in 100 equal parts. Here we use household_id
    + Each chunk (0-99) will be read in 1000 rows at a time with `pd.read_sql_query(""" select [...] where chunck = """ + str(chunk) + ';' , conn, chunksize= 1000):`
3. Build URLs for the OSRM server 
    + We just need to build a list of string `'http://OSRM IP/route/v1/driving/' + Long/lat ` to pass to our OSRM server
4. To speed up http requests call we use the **asyncio** package. https://docs.python.org/3/library/asyncio.html and https://pawelmhm.github.io/asyncio/python/aiohttp/2016/04/22/asyncio-aiohttp.html
    + Allows us to send requests to the OSRM server without waiting on an response. 
5. We get back a json object from OSRM, we parse this and upload it back to the SQL server (I also write a csv file for long term storage) 


In [ ]:
#del cursor

#Netezza
conn = pyodbc.connect(
    'DRIVER={XXXX};'
    'UID=XXXX;'
    'SERVER=XXXX;'
    'DATABASE=XXXX;'
    'PWD=XXXX;'
    )

conn2 = pyodbc.connect(
    'DRIVER={XXXX};'
    'UID=XXXX;'
    'SERVER=XXXX;'
    'DATABASE=XXXX;'
    'PWD=XXXX;'
    )
cursor = conn.cursor()

cursor2 = conn2.cursor()


#Functions
#ToDo
## Server IP and Long/Lat colum names as user input as user inpuut
def create_urls(input_table):
    url_list = []
    for index, row in input_table.iterrows():
        url ='http://IP:PORT/route/v1/driving/' + str(row['LONGITUDE_ADDR']) + ',' + str(row['LATITUDE_ADDR']) + ';' + str(row['LONGITUDE_AUSA']) + ',' + str(row['LATITUDE_AUSA']) + "?overview=simplified"
        url_list.append(url)
    
    return url_list

####### OSRM calls
async def fetch(url, session):
    async with session.get(url) as response:
        return await response.json()
        
async def run():
    
    tasks = []
   
    
    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
       
            try:
                for i in urls:
                    task = asyncio.ensure_future(fetch(i, session))

                    tasks.append(task)

                responses = await asyncio.gather(*tasks)
                return(responses)
            except:
               
                sys.exit("Error: at" + chunk)
        # you now have all response bodies in this variable
        
    


#We loop over chuncks of 1000 in our sql datafram
for chunk in range(97, 100):
    print(chunk)
    counter = 0 
    for df in pd.read_sql_query(""" select HOUSEHOLD_ID, LATITUDE_ADDR, LONGITUDE_ADDR, AUSA_STORE_ID,
    LATITUDE_AUSA, LONGITUDE_AUSA ,CURRENT_DATE as time_stamp from ahutstanl..hh_pref_store where chunck = """ + str(chunk) + ';' , conn, chunksize= 1000):

        counter= counter+1
        #print(counter)
        #create urls
        urls = create_urls(df)       

        #Get drivetime    
        loop = asyncio.get_event_loop()
        future = asyncio.ensure_future(run())
        stime = time.time()
        responses = loop.run_until_complete(future)
        dur = time.time() - stime
        print("Calculated %d distances in %.2f seconds: %.0f per second" % (len(responses),
                                                                                        dur,
                                                                                       len(responses)/dur))
        #Parse json object 
        final = []
        for i in responses:
            if i['code'] == 'Ok':
                status = i['code']
                tot_time_s = i['routes'][0]['duration']
                tot_dist_m = i['routes'][0]['distance']
                out =  [status,tot_time_s,tot_dist_m]

            else:
                status = i['code']
                tot_time_s = 999999
                tot_dist_m = 999999
                out =  [status,tot_time_s,tot_dist_m]

            final.append(out)


        df1 = pd.DataFrame(final)
        df_c = pd.concat([df1.reset_index(drop=True), df], axis=1)
        #df_c.iloc[:,[0,1,2,3,4,9,10]].round(2).to_csv('out/output_osrm_' + str(counter) + '.csv', encoding='utf-8', index=False,header  = False )
        df_c.to_csv('out/output_osrm_' + str(counter) + '_'+ str(chunk) + '.csv', encoding='utf-8', index=False, sep = '|' , header  = False )

        #Insert into sql
        cursor2.execute("""  INSERT INTO ahutstanl..ausa_drive_tim_hh
                  SELECT * FROM
                  EXTERNAL 'C:/Users/JKrueger/Documents/Scripts/Python_notebook/out/output_osrm_"""  + str(counter) + '_'+ str(chunk) + """.csv'
                  USING
                  (   LOGDIR 'C:/Users/JKrueger/Documents/Scripts/Python_notebook/'
                  DELIMITER '|'
                  QUOTEDVALUE 'DOUBLE'
                  Y2BASE 2000
                  ENCODING 'internal'
                  REMOTESOURCE 'ODBC'
                  ESCAPECHAR '\'
                  )     """)
        
       
        
        conn2.commit()

